# Get Raw Images

This notebook will demonstrate how to search for a specific image in order to get its `sequence_id`.

Using a `sequence_id` we will create an `ObservationInfo` object for discovering information about the observation before showing how to download the image metadata and FITS files.

In [1]:
import pandas as pd
from astropy.coordinates import SkyCoord

from panoptes.data.search import search_observations
from panoptes.data.observations import ObservationInfo

/home/wtgee/miniconda3/envs/panoptes-data/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Search for observations

Using a sky coordinate, search for all matching observations with a certain radius.

In [2]:
search_coords = SkyCoord.from_name('M42')
search_coords

<SkyCoord (ICRS): (ra, dec) in deg
    (83.82208, -5.39111)>

In [3]:
search_results = search_observations(
    coords=search_coords,
    radius=10,
    min_num_images=5,
    unit_id='PAN019'
)

print(f'Found {len(search_results)} observations')

Found 20 observations


In [4]:
# Show a random example (run this cell again to see a different entry)
search_results.sample(1).T

,1055
camera_id,c623e9
camera_lens_serial_number,RL1552832
camera_serial_number,273074013304
coordinates_airmass,1.136313
coordinates_mount_dec,-5.39111
coordinates_mount_ha,23.101843
coordinates_mount_ra,83.82208
field_name,M42
imageh,0
imagew,0


In [5]:
# Show the first and last five entries.
search_results

,camera_id,camera_lens_serial_number,camera_serial_number,coordinates_airmass,coordinates_mount_dec,coordinates_mount_ha,coordinates_mount_ra,field_name,imageh,imagew,num_images,sequence_id,software_version,time,total_exptime,unit_id,exptime
1064,42433a,RL1548256,273074013260,1.103328,-5.39111,23.886201,83.82208,M42,0,0,65,PAN019_42433a_20220114T063812,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-14 06:38:12+00:00,5850.0,PAN019,90.000000
1063,c623e9,RL1552832,273074013304,1.111012,-5.39111,0.479317,83.82208,M42,0,0,64,PAN019_c623e9_20220114T063812,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-14 06:38:12+00:00,5696.0,PAN019,89.000000
1062,42433a,RL1548256,273074013260,1.298554,-5.39111,2.106934,83.82208,M42,0,0,90,PAN019_42433a_20220114T085722,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-14 08:57:22+00:00,8100.0,PAN019,90.000000
1061,c623e9,RL1552832,273074013304,1.116015,-5.39111,0.600016,83.82208,M42,0,0,90,PAN019_c623e9_20220114T085722,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-14 08:57:22+00:00,8012.0,PAN019,89.022222
1060,42433a,RL1548256,273074013260,1.165687,-5.39111,22.777680,83.82208,M42,0,0,59,PAN019_42433a_20220115T050238,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 05:02:38+00:00,5303.0,PAN019,89.881356
1059,c623e9,RL1552832,273074013304,1.511834,-5.39111,21.191126,83.82208,M42,0,0,59,PAN019_c623e9_20220115T050238,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 05:02:38+00:00,5251.0,PAN019,89.000000
1056,42433a,RL1548256,273074013260,1.102712,-5.39111,23.977197,83.82208,M42,0,0,33,PAN019_42433a_20220115T070915,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 07:09:15+00:00,2970.0,PAN019,90.000000
1055,c623e9,RL1552832,273074013304,1.136313,-5.39111,23.101843,83.82208,M42,0,0,33,PAN019_c623e9_20220115T070915,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 07:09:15+00:00,2937.0,PAN019,89.000000
1054,42433a,RL1548256,273074013260,1.266788,-5.39111,1.950433,83.82208,M42,0,0,105,PAN019_42433a_20220115T082108,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 08:21:08+00:00,9451.0,PAN019,90.009524
1053,c623e9,RL1552832,273074013304,1.318294,-5.39111,2.195532,83.82208,M42,0,0,105,PAN019_c623e9_20220115T082108,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-15 08:21:08+00:00,9347.0,PAN019,89.019048


## Select observation

A `sequence_id` is used to explore observations. Here we take the most recent observation to get the `sequence_id`.

In [6]:
# We choose an entry that only has a few images while running on binder and get the most recent.
record = search_results.query('num_images < 15').iloc[-1]
record.T

camera_id                                                   42433a
camera_lens_serial_number                                RL1548256
camera_serial_number                                  273074013260
coordinates_airmass                                       1.455817
coordinates_mount_dec                                     -5.39111
coordinates_mount_ha                                      2.687999
coordinates_mount_ra                                      83.82208
field_name                                                     M42
imageh                                                           0
imagew                                                           0
num_images                                                      12
sequence_id                          PAN019_42433a_20220117T104417
software_version             POCSv0.8.0.post0.dev7+g8c57c125.dirty
time                                     2022-01-17 10:44:17+00:00
total_exptime                                               10

## Get Observation info

We can get more information about the observation by creating an `ObservationInfo` object. You can create it with just a `sequence_id` (e.g. `ObservationInfo(sequence_id=record.sequence_id)) but here we pass all the metadata.

In [7]:
obs_info = ObservationInfo(meta=record)

# Show the observation object.
obs_info

Obs: seq_id=PAN019_42433a_20220117T104417 num_images=11

In [8]:
obs_info.meta

camera_id                                                   42433a
camera_lens_serial_number                                RL1548256
camera_serial_number                                  273074013260
coordinates_airmass                                       1.455817
coordinates_mount_dec                                     -5.39111
coordinates_mount_ha                                      2.687999
coordinates_mount_ra                                      83.82208
field_name                                                     M42
imageh                                                           0
imagew                                                           0
num_images                                                      12
sequence_id                          PAN019_42433a_20220117T104417
software_version             POCSv0.8.0.post0.dev7+g8c57c125.dirty
time                                     2022-01-17 10:44:17+00:00
total_exptime                                               10

### Show example of the image metadata and how to save to file

The metadata for each image is stored in the `obs_info.image_metadata` dataframe. 

We view a random example and then write the metadata to a file that can be downloaded in the file browser on the left.

In [9]:
obs_info.image_metadata

,time,file_creation_date,uid,unit_id,sequence_id,raw_url,status,MJD_OBS,environment_moonsep,environment_moonfrac,...,location_elevation,ha,dec,delta_seconds,delta_ha_arcsec,delta_ra_arcsec,delta_dec_arcsec,ha_drift_arcsec_per_sec,ra_drift_arcsec_per_sec,dec_drift_arcsec_per_sec
1,2022-01-17 10:47:23+00:00,2022-01-17 10:47:23+00:00,PAN019_42433a_20220117T104417_20220117T104723,PAN019,PAN019_42433a_20220117T104417,https://storage.googleapis.com/panoptes-images...,SOLVED,59596.034711,43.601980,0.996071,...,3400.0,40.784600,-1.506587,125.0,1891.567846,-11.395324,-17.449204,0.092543,-0.091130,-0.139594
2,2022-01-17 10:49:28+00:00,2022-01-17 10:49:28+00:00,PAN019_42433a_20220117T104417_20220117T104928,PAN019,PAN019_42433a_20220117T104417,https://storage.googleapis.com/panoptes-images...,SOLVED,59596.036157,43.603061,0.996085,...,3400.0,41.307428,-1.510995,125.0,1882.180673,-2.011851,-15.873018,0.017445,-0.016089,-0.126984
3,2022-01-17 10:51:31+00:00,2022-01-17 10:51:31+00:00,PAN019_42433a_20220117T104417_20220117T105131,PAN019,PAN019_42433a_20220117T104417,https://storage.googleapis.com/panoptes-images...,SOLVED,59596.037581,43.604125,0.996098,...,3400.0,41.825226,-1.515136,123.0,1864.072509,-13.987257,-14.941550,0.115061,-0.113677,-0.121476
4,2022-01-17 10:53:36+00:00,2022-01-17 10:53:36+00:00,PAN019_42433a_20220117T104417_20220117T105336,PAN019,PAN019_42433a_20220117T104417,https://storage.googleapis.com/panoptes-images...,SOLVED,59596.039028,43.605207,0.996112,...,3400.0,42.353025,-1.519168,125.0,1900.077220,-19.910322,-14.558477,0.160618,-0.159225,-0.116468
5,2022-01-17 10:55:43+00:00,2022-01-17 10:55:43+00:00,PAN019_42433a_20220117T104417_20220117T105543,PAN019,PAN019_42433a_20220117T104417,https://storage.googleapis.com/panoptes-images...,SOLVED,59596.040486,43.606306,0.996126,...,3400.0,42.883861,-1.523189,127.0,1911.010791,-0.762724,-14.478144,0.007329,-0.006004,-0.114001
6,2022-01-17 10:57:47+00:00,2022-01-17 10:57:47+00:00,PAN019_42433a_20220117T104417_20220117T105747,PAN019,PAN019_42433a_20220117T104417,https://storage.googleapis.com/panoptes-images...,SOLVED,59596.041933,43.607388,0.996140,...,3400.0,43.403188,-1.527697,124.0,1869.577255,-4.448575,-16.237728,0.037236,-0.035863,-0.130949
7,2022-01-17 10:59:58+00:00,2022-01-17 10:59:58+00:00,PAN019_42433a_20220117T104417_20220117T105958,PAN019,PAN019_42433a_20220117T104417,https://storage.googleapis.com/panoptes-images...,SOLVED,59596.043449,43.608523,0.996155,...,3400.0,43.955122,-1.532015,131.0,1986.961980,-16.546530,-15.580074,0.127649,-0.126263,-0.118932
8,2022-01-17 11:02:07+00:00,2022-01-17 11:02:07+00:00,PAN019_42433a_20220117T104417_20220117T110207,PAN019,PAN019_42433a_20220117T104417,https://storage.googleapis.com/panoptes-images...,SOLVED,59596.044942,43.609638,0.996169,...,3400.0,44.496383,-1.536147,129.0,1948.539059,-8.207563,-14.896005,0.064954,-0.063601,-0.115473
9,2022-01-17 11:04:12+00:00,2022-01-17 11:04:12+00:00,PAN019_42433a_20220117T104417_20220117T110412,PAN019,PAN019_42433a_20220117T104417,https://storage.googleapis.com/panoptes-images...,SOLVED,59596.046389,43.610721,0.996183,...,3400.0,45.019356,-1.540324,125.0,1882.702321,-2.535088,-15.044035,0.021619,-0.020273,-0.120352
10,2022-01-17 11:06:22+00:00,2022-01-17 11:06:22+00:00,PAN019_42433a_20220117T104417_20220117T110622,PAN019,PAN019_42433a_20220117T104417,https://storage.googleapis.com/panoptes-images...,SOLVED,59596.047894,43.611846,0.996197,...,3400.0,45.565891,-1.544539,130.0,1967.524883,-12.151451,-15.201941,0.094807,-0.093438,-0.116938


In [10]:
# Randomly sample one image from the dataframe then transpose it so it's easier to read.
obs_info.image_metadata.sample(1).T

,4
time,2022-01-17 10:53:36+00:00
file_creation_date,2022-01-17 10:53:36+00:00
uid,PAN019_42433a_20220117T104417_20220117T105336
unit_id,PAN019
sequence_id,PAN019_42433a_20220117T104417
raw_url,https://storage.googleapis.com/panoptes-images...
status,SOLVED
MJD_OBS,59596.039028
environment_moonsep,43.605207
environment_moonfrac,0.996112


In [10]:
# obs_info.image_metadata.to_csv(f'{obs_info.sequence_id}-image-metadata.csv')

### Download files locally

We can use the built-in `download_images` method of our `obs_info` object to download the files locally.

By default this will create an output directory that is the same as the `sequence_id` but this can be changed. You can also pass a limited list of files for download. 

Run `obs_info.download_images?` for more information.

In [11]:
# image_files = obs_info.download_images()

### Write image list to a CSV file, use `wget` to download

Or, if you prefer you can just get a list of files saved as csv and work on them in a command line terminal.

This first creates a DataFrame then immediately writes that to a CSV file. There are many ways to accomplish this but this is an easy way.

We then view the last few lines of the file using the command line tool `tail` called in a cell with the `!` operator.

We then use the same `!` operator to call `wget` in the shell and download a list of files. Note that you can just copy the CSV file to your computer or server and use the `wget -i <filename.txt>` command to download all the files. **Note that this can be a lot of data!**

In [12]:
# Create a dataframe and then write it to a CSV file.
pd.DataFrame(obs_info.get_image_list()).to_csv(f'{obs_info.sequence_id}-raw.txt', index=False, header=False)

In [13]:
# View the last entries in the file.
!tail {obs_info.sequence_id}-raw.txt

https://storage.googleapis.com/panoptes-images-raw/PAN019/42433a/20220117T104417/20220117T104928.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/42433a/20220117T104417/20220117T105131.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/42433a/20220117T104417/20220117T105336.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/42433a/20220117T104417/20220117T105543.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/42433a/20220117T104417/20220117T105747.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/42433a/20220117T104417/20220117T105958.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/42433a/20220117T104417/20220117T110207.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/42433a/20220117T104417/20220117T110412.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN019/42433a/20220117T104417/20220117T110622.fits.fz
https://storage.googleapis.com/panoptes-images-raw/PAN0

#### Download the data

In [14]:
### Uncomment to run ###
# !wget -i {obs_info.sequence_id}-raw.txt
